In [1]:
#choose a time frame for the whole notebook
time_frame = 89

# PCA of the head to find the eigenvectors

In [2]:
# Import packages
import cv2
import cc3d #connected components in 3D
import numpy as np
import pandas as pd
from skimage import io, morphology
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation
%matplotlib notebook

ModuleNotFoundError: No module named 'cv2'

In [3]:
# Define functions
def read(filename):
    volume_time = io.imread(filename)
    t,z,x,y = volume_time.shape
    return volume_time,t,z,x,y

def connected_component_label_2D(img):
    # Applying connected components
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(img)
    # Map component labels to hue val, 0-179 is the hue range in OpenCV
    label_hue = np.uint8(179*labels/np.max(labels))
    blank_ch = 255*np.ones_like(label_hue)
    labeled_img = cv2.merge([label_hue, blank_ch, blank_ch])
    # Converting cvt to BGR
    labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2BGR)
    # set bg label to black
    labeled_img[label_hue==0] = 0
    return labeled_img

# https://www.datacamp.com/community/tutorials/matplotlib-3d-volumetric-data
def remove_keymap_conflicts(new_keys_set):
    for prop in plt.rcParams:
        if prop.startswith('keymap.'):
            keys = plt.rcParams[prop]
            remove_list = set(keys) & new_keys_set
            for key in remove_list:
                keys.remove(key)

def multi_slice_viewer(volume):
    remove_keymap_conflicts({'j', 'k'})
    fig, ax = plt.subplots()
    ax.volume = volume
    ax.index = volume.shape[0] // 2
    cc_volume = connected_component_label_2D(volume[ax.index])
    ax.imshow(cc_volume, cmap='gray')
    ax.set_title('z=%i/%i' % (ax.index+1, z))
    fig.canvas.mpl_connect('key_press_event', process_key)

def process_key(event):
    fig = event.canvas.figure
    ax = fig.axes[0]
    if event.key == 'j':
        previous_slice(ax)
    elif event.key == 'k':
        next_slice(ax)
    fig.canvas.draw()

def previous_slice(ax):
    volume = ax.volume
    ax.index = (ax.index - 1) % volume.shape[0]  # wrap around using %
    cc_volume = connected_component_label_2D(volume[ax.index])
    ax.images[0].set_array(cc_volume)
    ax.set_title('z=%i/%i' % (ax.index+1, z))

def next_slice(ax):
    volume = ax.volume
    ax.index = (ax.index + 1) % volume.shape[0]
    cc_volume = connected_component_label_2D(volume[ax.index])
    ax.images[0].set_array(cc_volume)
    ax.set_title('z=%i/%i' % (ax.index+1, z))

### Read the data of the segmented head and whole cell

In [4]:
# t, z, y, x are the same for the head and whole cell segmentations
_, t, z, y, x = read('segmentation/sperm00068_t1_146_segmentation_head.tif')

print('Slices in Z:', z)
print('Time steps:', t)
print()
print("Rows or Height of frame:", y)
print("Columns or Width of frame:", x)

NameError: name 'io' is not defined

In [ ]:
volume_time_head = io.imread('segmentation/sperm00068_t1_146_segmentation_head.tif')
volume_time_cell = io.imread('segmentation/sperm00068_t1_146_segmentation_cell.tif')

### Connected components in 2D

https://www.pyimagesearch.com/2021/02/22/opencv-connected-component-labeling-and-analysis/

In [ ]:
multi_slice_viewer(volume_time_head[time_frame])

### Connected components in 3D

https://github.com/seung-lab/connected-components-3d

To change the size of the z axis:
https://stackoverflow.com/questions/13685386/matplotlib-equal-unit-length-with-equal-aspect-ratio-z-axis-is-not-equal-to

In [ ]:
labels = cc3d.connected_components(volume_time_cell[time_frame])

fig = plt.figure()
ax = plt.axes(projection ='3d')
ax.set_box_aspect([1,1,0.5])

for l in range(1, np.max(labels)+1): #assuming label=0 is always the background
    k, j, i = np.where(labels==l) #k refers to the z direction, j to the y and i to the x
    j = y-j #The y axis is inverted with respect to how it is seen when printing the image (bc imshow flips the y axis)
    ax.plot3D(i, j, k, 'o', label='Label %i'%l)

ax.legend()
ax.set_xlabel('x'); ax.set_ylabel('y'); ax.set_zlabel('z')
ax.set_xlim(0,x); ax.set_ylim(0,y)
ax.set_title('Connected components in 3D')
plt.show()

In [ ]:
# Choose the label we want to do the PCA of
k, j, i = np.where(labels==1) #k refers to the z direction, j to the y and i to the x
j = y-j

## PCA

In [ ]:
X = np.array([i, j, k])

covMatrix = np.cov(X, bias=True) #bias is True to calculate the population covariance and not the sample one
eigval, eigvec = np.linalg.eig(covMatrix)
print(eigval)
print(eigvec) #each column is a new vector --> a row is all the i

# The vectors shown are the eigenvectors of the covariance matrix scaled by the square root of the corresponding
# eigenvalue, and shifted so their tails are at the mean.
vec = eigvec * np.sqrt(eigval)
print(vec)

In [ ]:
origin = np.mean(X, axis=1)

# Show the vectors
fig = plt.figure()
ax = plt.axes(projection ='3d')
ax.plot3D(i, j, k, 'o')
ax.quiver(*origin, vec[0,:], vec[1,:], vec[2,:], color='k')
ax.set_xlabel('x'); ax.set_ylabel('y'); ax.set_zlabel('z')
plt.show()

In [ ]:
# length of each vector
print(np.sqrt(np.sum((vec)**2, axis=0))) #should give the same result as
print(np.sqrt(eigval)) #because the eigenvectors are unitary

# Finding the midpiece center of mass

By dilating the head two times and seeing where there is cell, we should be able to find the neck and it's CoM.

In [ ]:
time_frame = 89
z_frame = 4

img_head = volume_time_head[time_frame][z_frame]
img_head = img_head.astype('float64')
img_head[np.where(img_head>0)]=1

img_cell = volume_time_cell[time_frame][z_frame]
img_cell = img_cell.astype('float64')
img_cell[np.where(img_cell>0)]=1

# Get a Cross Shaped Kernel
kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (3,3))

dilation1 = cv2.dilate(img_head,kernel,iterations = 5)
dilation2 = cv2.dilate(img_head,kernel,iterations = 10)
dilation3 = cv2.dilate(img_head,kernel,iterations = 15)

In [ ]:
band1 = dilation2 - dilation1 + img_cell
band2 = dilation3 - dilation2 + img_cell

y1,x1 = np.mean(np.where(band1==2), axis=1)
y2,x2 = np.mean(np.where(band2==2), axis=1)

plt.imshow(band1, cmap='gray')
#plt.imshow(band2, cmap='gray')
#plt.quiver(x1, y1, x2-x1, y2-y1, color='b')
plt.plot(x1, y1, 'r.')
plt.plot(x2, y2, 'b.')
plt.show()

### Dilating in 3D

In [ ]:
def find_neck(time_frame, img_head, img_cell):
    img_head = img_head.astype('bool')
    img_cell = img_cell.astype('bool')

    dilation1 = ndi.binary_dilation(img_head, morphology.ball(radius=2), iterations=4)
    dilation2 = ndi.binary_dilation(img_head, morphology.ball(radius=2), iterations=6)

    band1 = dilation2^dilation1
    z,y,x = np.mean(np.where(band1&img_cell==True), axis=1)
    
    return x,y,z

def find_tail(time_frame, img_head, img_cell):
    img_head = img_head.astype('bool')
    img_cell = img_cell.astype('bool')

    dilation1 = ndi.binary_dilation(img_head, morphology.ball(radius=2), iterations=6)
    dilation2 = ndi.binary_dilation(img_head, morphology.ball(radius=2), iterations=8)

    band1 = dilation2^dilation1
    z1,y1,x1 = np.mean(np.where(band1&img_cell==True), axis=1)
    
    return x1,y1,z1

In [ ]:
time_frame = 89

img_head = volume_time_head[time_frame]
img_cell = volume_time_cell[time_frame]

neck = find_neck(time_frame, img_head, img_cell)

### PCA for all the time frames

In [ ]:
# Create an animation for all the timeframes
# Segmented points in blue, their coordinate system in black and the neck in red
fig = plt.figure()
ax = plt.axes(projection ='3d')
ax.set_box_aspect([1,1,0.3])
history_x = []; history_y = []; history_z = []

def anim3D(time_frame):
    img_head = volume_time_head[time_frame]
    img_cell = volume_time_cell[time_frame]
    
    labels = cc3d.connected_components(img_head)
    if np.max(labels>0):
        k, j, i = np.where(labels==1) #k refers to the z direction, j to the y and i to the x
        j = y-j
        X = np.array([i, j, k])
        origin = np.mean(X, axis=1)
        
        x_neck,y_neck,z_neck = find_neck(time_frame, img_head, img_cell)
        x_tail,y_tail,z_tail = find_tail(time_frame, img_head, img_cell)
        history_x.append(x_neck); history_y.append(y_neck); history_z.append(z_neck)

        covMatrix = np.cov(X, bias=True) #bias is True to calculate the population covariance and not the sample one
        eigval, eigvec = np.linalg.eig(covMatrix)
        vec = eigvec * np.sqrt(eigval)
        
        if (time_frame%1==0):
            ax.clear()
            ax.text2D(0.05, 0.95, 't = %i' % time_frame, transform=ax.transAxes) #placement (0,0) would be the bottom left, (0,1) would be the top left
            ax.plot3D(i, j, k, 'o')
            ax.plot3D(y_neck, x_neck, z_neck, 'ro')
            #ax.plot(history_x, history_y, history_z, color='orange') 
            #ax.scatter(x_tail, y_tail, z_tail, c='b')
            ax.quiver(*origin, vec[0,:], vec[1,:], vec[2,:], color='k')
            ax.set_xlabel('x'); ax.set_ylabel('y'); ax.set_zlabel('z')
            ax.set_xlim(0,x); ax.set_ylim(0,y); ax.set_zlim(0,z)
            plt.show()
    
    return 0

ani = animation.FuncAnimation(fig, anim3D, t+1, interval=100, repeat=False, blit=True)

### In the head's coordinate system

In [ ]:
def cartesian_to_spherical(x,y,z):
    r = np.sqrt(x**2+y**2+z**2)
    if x>0:
        phi = np.arctan(y/x)
    elif x<0:
        phi = np.arctan(y/x) + np.pi
    else:
        phi = np.pi/2 
    theta = np.arccos(z/r)
    return r,phi,theta

def spherical_to_cartesian(r,phi,theta):
    x = r*np.sin(theta)*np.cos(phi)
    y = r*np.sin(theta)*np.sin(phi)
    z = r*np.cos(theta)
    return x,y,z

def plot_in_head_coord(origin, eigvec, p):
    # First we calculate the spherical coordinates of the largest axis of the head
    x,y,z = eigvec[:,0] #First eigenvector, we calculate the rotation always with it 
    r,phi,theta = cartesian_to_spherical(x,y,z)
    print(f'orig: {r}, {phi}, {theta}')

    # Then, we calculate the distance of the point to the new set of axis and the spherical coordinates of the point
    d = p - origin
    r_p,phi_p,theta_p = cartesian_to_spherical(*d)
    print(f'end: {r_p}, {phi_p}, {theta_p}')
    
    # Now, we calculate the new coordinates
    x_p,y_p,z_p = spherical_to_cartesian(r_p,-phi+phi_p,-theta+theta_p)

    return x_p,y_p,z_p

In [ ]:
# Create an animation for all the timeframes of the neck in the head's coordinate system
fig = plt.figure()
ax = plt.axes(projection ='3d')
ax.set_box_aspect([1,1,0.3])
history_x = []; history_y = []; history_z = []

def anim3D(time_frame):
    img_head = volume_time_head[time_frame]
    img_cell = volume_time_cell[time_frame]
    
    labels = cc3d.connected_components(img_head)
    if np.max(labels>0):
        k, j, i = np.where(labels==1) #k refers to the z direction, j to the y and i to the x
        j = y-j
        X = np.array([i, j, k])
        origin = np.mean(X, axis=1)
        
        x_neck,y_neck,z_neck = find_neck(time_frame, img_head, img_cell)

        covMatrix = np.cov(X, bias=True) #bias is True to calculate the population covariance and not the sample one
        eigval, eigvec = np.linalg.eig(covMatrix)
        vec = eigvec * np.sqrt(eigval)
        
        R = eigvec
        p = neck - origin
        new_p = p@R
        
        history_x.append(new_p[0]); history_y.append(new_p[1]); history_z.append(new_p[2])
        
        if (time_frame%1==0):
            ax.clear()
            ax.text2D(0.05, 0.95, 't = %i' % time_frame, transform=ax.transAxes) #placement (0,0) would be the bottom left, (0,1) would be the top left
            ax.plot(*new_p, 'r.')
            ax.plot(history_x, history_y, history_z, color='orange', linestyle='-') 
            ax.set_xlabel('x'); ax.set_ylabel('y'); ax.set_zlabel('z')
            ax.set_xlim(-100,100); ax.set_ylim(-100,100); ax.set_zlim(-10,10)
            plt.show()
    
    return 0

ani = animation.FuncAnimation(fig, anim3D, t+1, interval=100, repeat=False, blit=True)

In [ ]:
orig = np.array([0,0,0])
vec = np.array([1,0,0])
p = np.array([1,2,3])

fig = plt.figure()
ax = plt.axes(projection ='3d')
ax.plot3D([0,vec[0]],[0,vec[1]], [0,vec[2]], 'k-')
ax.plot3D([0,0],[0,1], [0,0], 'k-')
ax.plot3D([0,0],[0,0], [0,1], 'k-')
ax.plot3D(*p, 'bo')
x,y,z = plot_in_head_coord(orig, vec, p)
ax.plot(x,y,z, 'ro')
ax.set_xlabel('x'); ax.set_ylabel('y'); ax.set_zlabel('z')

In [ ]:
# Create an animation for all the timeframes in the head's coordinate system
# Segmented points in blue, their coordinate system in black and the neck in red
fig = plt.figure()
ax = plt.axes(projection ='3d')
ax.set_box_aspect([1,1,0.3])

def anim3D(time_frame):
    img_head = volume_time_head[time_frame]
    img_cell = volume_time_cell[time_frame]
    
    labels = cc3d.connected_components(img_head)
    if np.max(labels>0):
        k, j, i = np.where(labels==1) #k refers to the z direction, j to the y and i to the x
        j = y-j
        X = np.array([i, j, k])
        origin = np.mean(X, axis=1)
        
        neck = find_neck(time_frame, img_head, img_cell)
        tail = find_tail(time_frame, img_head, img_cell)        

        covMatrix = np.cov(X, bias=True) #bias is True to calculate the population covariance and not the sample one
        eigval, eigvec = np.linalg.eig(covMatrix)
        vec = eigvec * np.sqrt(eigval)
        
        R = eigvec
        p = neck - origin
        
        if (time_frame%1==0):
            ax.clear()
            ax.text2D(0.05, 0.95, 't = %i' % time_frame, transform=ax.transAxes) #placement (0,0) would be the bottom left, (0,1) would be the top left
            #ax.plot3D(i, j, k, 'o')
            #ax.scatter(*neck_head_coord, c='r')
            #ax.scatter(*tail_head_coord, c='b')
            ax.plot(*(p@R), 'bo')
            for l in range(3):
                ax.plot3D([0,R[0,l]], [0,R[1,l]], [0,R[2,l]])
            ax.set_xlabel('x'); ax.set_ylabel('y'); ax.set_zlabel('z')
            plt.show()
    
    return 0

ani = animation.FuncAnimation(fig, anim3D, t+1, interval=100, repeat=False, blit=True)